# Import modules

In [3]:
from utils import * 

import numpy as np
import pandas as pd
from pprint import pprint
import os
import matplotlib.pyplot as plt
from collections import defaultdict

# Gensim
from gensim.test.utils import datapath
from gensim.test.utils import common_texts, get_tmpfile

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.option_context('display.max_colwidth', 500);

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


config = get_config('config.yaml')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\enlik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\enlik\Miniconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [7]:
# setup input and output path
input_path = 'preprocessed_data/bolt/p1/'
output_path = 'std_lda_vis/bolt/p1/'

# Import dataset
df = pd.read_csv(config['csv_input_local']['bolt_apple_google_p1'], index_col=0)
df = df.reset_index(drop=True)
reviews = df.review
reviews

0        The first thing I noticed is that you can't pr...
1        Your GPS setting around Cape Town Internationa...
2        I was invited by my friend and was given a £10...
3        Your drivers are great BUT your support is no ...
4        Loving the app, but can only give it a medium ...
                               ...                        
40360    Its the 5th time Im saying, But When Will ther...
40361                                        Just love it!
40362                                           Great app!
40363                           Super convenient and fast!
40364    Taxify's purpose was that you can CHOOSE your ...
Name: review, Length: 40365, dtype: object

In [8]:
term_doc = pd.read_pickle(input_path + 'term_doc.pkl')
data_lemmatized = pd.read_pickle(input_path + 'data_lemmatized.pkl')
dictionary = pd.read_pickle(input_path + 'dictionary.pkl')
tf_idf = pd.read_pickle(input_path + 'tf_idf.pkl')

# <mark>Mallet's LDA model after tuning &#8592; Best Model
*The difference between Mallet and Gensim’s standard LDA is that Gensim uses a Variational Bayes sampling method which is faster but less precise that Mallet’s Gibbs Sampling.*  [link](https://towardsdatascience.com/basic-nlp-on-the-texts-of-harry-potter-topic-modeling-with-latent-dirichlet-allocation-f3c00f77b0f5)

In [28]:
# ldamallet = gensim.models.wrappers.LdaMallet.load(datapath("model"))
# ldamallet

In [4]:
# Solution on windows 10, fixing Java path on mallet.bat
# https://stackoverflow.com/questions/52845345/lda-mallet-calledprocesserror
# mallet_path = '../mallet-2.0.8/bin/mallet' # update this path

import os
os.environ.update({'MALLET_HOME':r'C:/mallet-2.0.8/'})

# mallet_path = 'mallet-2.0.8/bin/mallet.bat' # update this path
mallet_path = 'C:/mallet-2.0.8/bin/mallet.bat'

# ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, 
#                                              corpus = term_doc,
#                                              num_topics = 6, 
#                                              random_seed = 100,
#                                              id2word = dictionary,
#                                              alpha = 1.5)

ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=term_doc, num_topics=20, id2word=dictionary)

In [15]:
# ldamallet.save(datapath("model"))
# ldamallet = gensim.models.wrappers.LdaMallet.load(datapath("model"))

In [5]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=dictionary, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(3,
  [('customer', 0.13074914641494276),
   ('discount', 0.11648925487045592),
   ('charge', 0.10524201646917052),
   ('end', 0.03414340228961639),
   ('month', 0.029323157260494077),
   ('distance', 0.02851978308897369),
   ('morning', 0.025908817031532437),
   ('pay', 0.02510544286001205),
   ('care', 0.021088572002410124),
   ('arrival', 0.01948182365936935)]),
 (1,
  [('taxi', 0.14087947882736157),
   ('rate', 0.0873371335504886),
   ('day', 0.07023615635179153),
   ('application', 0.06616449511400652),
   ('app', 0.06596091205211727),
   ('fee', 0.056799674267100975),
   ('scooter', 0.03664495114006515),
   ('star', 0.0364413680781759),
   ('work', 0.016490228013029316),
   ('move', 0.016083061889250815)]),
 (12,
  [('card', 0.12560856864654332),
   ('money', 0.10399221032132425),
   ('company', 0.08490749756572541),
   ('account', 0.06523855890944498),
   ('payment', 0.05998052580331061),
   ('amount', 0.053554040895813046),
   ('change', 0.05102239532619279),
   ('cash', 0.045

## Visualize Mallet's LDA model

In [17]:
# Visualize the topics
model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, term_doc, dictionary)
vis


C:\Users\enlik\Miniconda3\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.095831  0.111252       1        1  5.139204
12    -0.082691  0.097849       2        1  5.099634
3     -0.089869  0.098742       3        1  5.071341
7     -0.150635 -0.172816       4        1  5.070551
0      0.265716 -0.005118       5        1  5.047910
5      0.341462 -0.026311       6        1  5.029768
16    -0.171465 -0.177979       7        1  5.025306
6     -0.075757  0.109280       8        1  5.020271
11     0.295406 -0.008954       9        1  5.016765
4     -0.121233  0.268882      10        1  5.015002
1     -0.148455 -0.123500      11        1  5.002140
17     0.331421 -0.024693      12        1  4.988482
15    -0.111502  0.269473      13        1  4.980894
18    -0.146226 -0.271452      14        1  4.979595
9      0.247554 -0.043326      15        1  4.978704
14    -0.194873 -0.256627      16        1  4.969597
2     -0.081641  0.099994      17        1  4.944011
19     0.124526 -0.110618      18        1  4.934939
8     -0.081281  0.117670      19        1  4.931714
10    -0.054627  0.048254      20        1  4.754172, topic_info=              Term         Freq        Total Category  logprob  loglift
4           driver  9884.000000  9884.000000  Default  30.0000  30.0000
42            ride  2775.000000  2775.000000  Default  29.0000  29.0000
0              app  5416.000000  5416.000000  Default  28.0000  28.0000
87            time  3285.000000  3285.000000  Default  27.0000  27.0000
10         service  2653.000000  2653.000000  Default  26.0000  26.0000
..             ...          ...          ...      ...      ...      ...
955  polite_driver    16.774431    26.255145  Topic20  -5.6763   2.5981
977        meeting    18.871234    33.248521  Topic20  -5.5585   2.4798
830           site    11.532421    30.541464  Topic20  -6.0510   2.0722
624     cars_alway     9.435617    26.367308  Topic20  -6.2517   2.0185
402          check    10.484019    99.081799  Topic20  -6.1463   0.8001

[1260 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
725       5  0.969118  aappointment
357      19  1.042392      able_use
1300      9  0.984038           abt
1570     10  0.976639   accept_trip
1831     16  0.946875    acceptance
...     ...       ...           ...
142       4  0.992954     yesterday
142       6  0.006848     yesterday
771      12  0.945626    younreport
801      11  0.994984          zone
1202     20  0.953833          zoom

[1667 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 13, 4, 8, 1, 6, 17, 7, 12, 5, 2, 18, 16, 19, 10, 15, 3, 20, 9, 11])

In [22]:
pyLDAvis.save_html(vis, 'mallet_lda_vis/bolt/mallet_lda_topics=20.html')

In [8]:
coherence_model_m = CoherenceModel(model=model, texts=data_lemmatized, dictionary=dictionary, coherence='c_npmi')
coherence_model_m.get_coherence_per_topic()

[-0.3059867099004702,
 -0.12998080224814468,
 -0.37507692936319176,
 -0.12106244239262642,
 -0.1672011091833666,
 -0.2817028690959776,
 -0.08394729728680743,
 -0.03950294631033207,
 -0.30251018034246346,
 -0.24172196958296335,
 -0.3961081712493524,
 -0.3007240772019151,
 -0.0458558785635095,
 -0.0317091176755673,
 -0.2018578473462231,
 -0.19911268613312452,
 -0.14421412777772974,
 -0.32787023523501235,
 -0.2992439538283151,
 -0.2621733651564374]

In [23]:
# model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)
model.top_topics(corpus = term_doc,topn=10)

[([(0.09835423197492163, 'support'),
   (0.05427115987460815, 'email'),
   (0.04760971786833856, 'journey'),
   (0.045454545454545456, 'complaint'),
   (0.03879310344827586, 'reason'),
   (0.03702978056426332, 'week'),
   (0.03487460815047022, 'refund'),
   (0.026057993730407524, 'customer_service'),
   (0.02586206896551724, 'reply'),
   (0.0219435736677116, 'team')],
  -3.6827256709456178),
 ([(0.12560856864654332, 'card'),
   (0.10399221032132425, 'money'),
   (0.08490749756572541, 'company'),
   (0.06523855890944498, 'account'),
   (0.05998052580331061, 'payment'),
   (0.053554040895813046, 'amount'),
   (0.05102239532619279, 'change'),
   (0.04556962025316456, 'cash'),
   (0.04089581304771178, 'pay'),
   (0.021616358325219083, 'card_payment')],
  -3.8655085738900197),
 ([(0.1270185112248917, 'issue'),
   (0.10476565576998818, 'app'),
   (0.05966916108704214, 'response'),
   (0.05907837731390311, 'today'),
   (0.052185899960614415, 'thing'),
   (0.04568727845608507, 'update'),
   (0

## For each topic, we could look at frequent and relevant words

In [9]:
frequencies = ldamallet.word_topics.sum(axis=0)
p_word = frequencies / ldamallet.word_topics.sum()
p_word_given_topic = ldamallet.word_topics / np.sum(ldamallet.word_topics, axis = 0)

lamda = 0.5
relevance = lamda * p_word_given_topic + (1-lamda) * p_word_given_topic / p_word 

C:\Users\enlik\AppData\Local\Temp/ipykernel_10832/1698140663.py:3: RuntimeWarning: invalid value encountered in true_divide
  p_word_given_topic = ldamallet.word_topics / np.sum(ldamallet.word_topics, axis = 0)


In [21]:
# topic_dict = {0.: "Platform/Device", 1.: "User Experience", 2.: "Value", 3.: "Service", 4.: "Trouble-shooting", 5.:"Shows"}

# for topic_id in range(6):
#     words = []
#     for id in np.argsort(relevance[topic_id,])[::-1][:15]:
#         words.append(dictionary[id])
#     print(f"Topic: {topic_dict[topic_id]}")
#     print(words)
#     print('\n')

In [22]:
np.argsort(relevance[0,])[::-1][:15]


array([1826, 1540, 4588,  270, 4624, 2459,  279, 5575, 2537,  306, 1493,
       7027,  635,  321, 4734], dtype=int64)

In [23]:
dictionary[1826]

'saw'

In [24]:
topic_dict = {0.: "Platform/Device", 1.: "User Experience", 2.: "Value", 3.: "Service", 4.: "Trouble-shooting", 5.:"Shows"}

for topic_id in range(6):
    words = []
    for id in np.argsort(relevance[topic_id,])[::-1][:15]:
        words.append(dictionary[id])
    print(f"Topic: {topic_dict[topic_id]}")
    print(words)
    print('\n')

Topic: Platform/Device
['saw', 'think', 'around', 'thank', 'out', 'everyone', 'well', 'must', 'wonder', 'self', 'course', 'sometime', 'value', 'up', 'allow']


Topic: User Experience
['think', 'r', 'value', 'well', 'brief', 'noone', 'known', 'more', 'welcome', 'something', 'edu', 'thank', 'down', 'everyone', 'going']


Topic: Value
['going', 'com', 'everyone', 'try', 'twice', 'nothing', 'tell', 'become', 'want', 'see', 'go', 'everything', 'over', 'willing', 'do']


Topic: Service
['out', 'consider', 'up', 'another', 'again', 'around', 'come', 'sub', 'sorry', 'self', 'anyway', 'plus', 'wonder', 'looking', 'allow']


Topic: Trouble-shooting
['sometime', 'thanx', 'more', 'last', 'th', 'around', 'brief', 'nothing', 'as', 'again', 'here', 'up', 'furthermore', 'well', 'thank']


Topic: Shows
['second', 'allow', 'wish', 'take', 'etc', 'on', 'everyone', 'indicate', 'away', 'would', 'anyway', 'took', 'more', 'brief', 'like']




## Interpret the topic model
1. Finding the dominant topic in each document
2. Find the most representative document for each topic
3. Topic distribution across documents
**The code used here for interpretation of the model are based on this website with a little modification by the user myself: <br>
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling**

### <mark>Mallet Model - Interpret the model</mark>

In [29]:
# ldamallet = gensim.models.wrappers.LdaMallet.load(datapath('model'))
model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)

In [30]:
df_topic_sents_keywords_m = format_topics_sentences(model, term_doc, data_lemmatized, df)

Getting main topic for document...
0  1000  2000  3000  4000  5000  6000  7000  8000  9000  10000  11000  12000  13000  14000  15000  16000  17000  18000  19000  20000  21000  22000  23000  24000  25000  26000  27000  28000  29000  30000  31000  32000  33000  34000  35000  36000  37000  38000  39000  40000  

In [31]:
all_topics = model.get_document_topics(bow = term_doc)

In [32]:
# df_topic_sents_keywords_m[df_topic_sents_keywords_m[0]]
df_topic_sents_keywords_m

,Dominant_Topic,Perc_Contribution,Keywords,0
0,0.0,0.0859,"car, driver, passenger, city, fact, street, ex...","[thing, prenook, trip, catch, flight, pm, toda..."
1,1.0,0.0924,"taxi, rate, day, application, app, fee, scoote...","[need, section, airport, spot, number, app]"
2,13.0,0.1102,"support, email, journey, complaint, reason, we...","[friend, use, would_allow, journey, support, c..."
3,13.0,0.0776,"support, email, journey, complaint, reason, we...","[driver, support, computer, response, case, re..."
4,17.0,0.1186,"driver, rider, promo, rating, case, quality, a...","[rating, moment, driver, rating, trip, driver,..."
...,...,...,...,...
40360,15.0,0.0711,"time, route, drivers_alway, drive, traffic, co...","[time, nonsense, driver, drive, support, usless]"
40361,10.0,0.0686,"love, service, friendly_driver, nice_one, good...",[love]
40362,0.0,0.0500,"car, driver, passenger, city, fact, street, ex...",[]
40363,17.0,0.0618,"driver, rider, promo, rating, case, quality, a...",[super_convenient]


In [33]:
# save to pickle file - df_topic_sents_keywords_m
import pickle

output_path = 'lda_mallet_model/'
with open(output_path + 'mallet_lda_df.pkl', 'wb') as f:
    pickle.dump(df_topic_sents_keywords_m, f)


In [34]:
# save to pickle file - all_reviews_p3.csv
import pickle

output_path = 'raw_data/'
with open(output_path + 'bolt_apple_google_p1.pkl', 'wb') as f:
    pickle.dump(df, f)

#### 1. Mallet LDA - Finding the dominant topic in each document

In [36]:
df_dominant_topic_m = find_dominant_topic_in_each_doc(df_topic_sents_keywords_m, df)
print("Finding the dominant topic in each document")
df_dominant_topic_m.head(10).style.set_properties(subset=['review'], **{'width': '400px'})

Finding the dominant topic in each document


,Dominant_Topic,Perc_Contribution,Keywords,Text,rating,review
0,0.000000,0.085900,"car, driver, passenger, city, fact, street, excuse, kind, turn, condition","['thing', 'prenook', 'trip', 'catch', 'flight', 'pm', 'today', 'booking', 'went', 'book', 'find', 'driver', 'book', 'service', 'get_rid', 'app', 'today']",1,The first thing I noticed is that you can't prenook your trip which isn't great. As I had to catch a flight at 4 pm today I decided to give it a go anyway using the instant booking. There were 2 drivers available near me and I went to book one of them. Completed the booking only to find that the driver disappeared and he now was shown as engaged. Went to book the other one and same happened again. Unreliable service. I will get rid of the app today
1,1.000000,0.092400,"taxi, rate, day, application, app, fee, scooter, star, work, move","['need', 'section', 'airport', 'spot', 'number', 'app']",4,"Your GPS setting around Cape Town International Airport needs to be looked at. It always take my location to the drop off and go section of the airport when I'm actually about 1km away from that spot. Sometimes the drivers mobile numbers does not match their profile. But other than that, the app is quite user friendly."
2,13.000000,0.110200,"support, email, journey, complaint, reason, week, refund, customer_service, reply, team","['friend', 'use', 'would_allow', 'journey', 'support', 'customer', 'promotion', 'contact', 'team', 'reply']",1,I was invited by my friend and was given a £10 free promo to use on my first trip. I tried to use it and it would not allow me so I booked my journey anyway. I called customer support and they simply said sorry nothing we can do. Not very customer friendly and really am disappointed because I downloaded it because of the free promotion. I did contact the team and they are of no help just like your reply
3,13.000000,0.077600,"support, email, journey, complaint, reason, week, refund, customer_service, reply, team","['driver', 'support', 'computer', 'response', 'case', 'response', 'support', 'discount', 'account', 'thought', 'client']",3,"Your drivers are great BUT your support is no good. Why can't it be like Uber where you can actually speak to someone. Instead of getting computer generated responses or like in my case, no response at all. The three stars are for the support. Also I have received a single discount to this account since I installed it three months ago. I'm very disappointed in Bolt, I thought they were there for their clients."
4,17.000000,0.118600,"driver, rider, promo, rating, case, quality, attitude, moment, cancelled_trip, profile","['rating', 'moment', 'driver', 'rating', 'trip', 'driver', 'apartment', 'ride', 'drive', 'destination']",3,"Loving the app, but can only give it a medium rating at the moment as the drivers you give a bad rating on still gets your trip. I had a bad experience with a driver outside my apartment, but he always gets my rides as he's parked there when with my one star and bad review (refused to drive to the destination, rude and aggressive)."
5,16.000000,0.096600,"app, phone, number, code, call, message, contact, datum, download, detail","['price', 'try', 'sister', 'invitation', 'code', 'code', 'reply']",2,The prices are good and the service is a great when I try it with my sister but when I register the invitation code became inactive. They ask me to change the payment method but the code stay inactive. I contacted them via the app but no reply.
6,17.000000,0.084500,"driver, rider, promo, rating, case, quality, attitude, moment, cancelled_trip, profile","['promotion', 'price', 'driver', 'rider', 'price', 'change', 'driver', 'destination', 'rider', 'thank', 'fare', 'taxi', 'pay']",3,"I like the promotion price. But some of the drivers frown as if we, the riders, caused the price to change. Some drivers call to ask of the destination of the riders before coming or not. I personally hate that! Thanks. What is the essence of picking Bolt if the fare wi

In [37]:
df_dominant_topic_m

,Dominant_Topic,Perc_Contribution,Keywords,Text,rating,review
0,0.0,0.0859,"car, driver, passenger, city, fact, street, ex...","[thing, prenook, trip, catch, flight, pm, toda...",1,The first thing I noticed is that you can't pr...
1,1.0,0.0924,"taxi, rate, day, application, app, fee, scoote...","[need, section, airport, spot, number, app]",4,Your GPS setting around Cape Town Internationa...
2,13.0,0.1102,"support, email, journey, complaint, reason, we...","[friend, use, would_allow, journey, support, c...",1,I was invited by my friend and was given a £10...
3,13.0,0.0776,"support, email, journey, complaint, reason, we...","[driver, support, computer, response, case, re...",3,Your drivers are great BUT your support is no ...
4,17.0,0.1186,"driver, rider, promo, rating, case, quality, a...","[rating, moment, driver, rating, trip, driver,...",3,"Loving the app, but can only give it a medium ..."
...,...,...,...,...,...,...
40360,15.0,0.0711,"time, route, drivers_alway, drive, traffic, co...","[time, nonsense, driver, drive, support, usless]",1,"Its the 5th time Im saying, But When Will ther..."
40361,10.0,0.0686,"love, service, friendly_driver, nice_one, good...",[love],5,Just love it!
40362,0.0,0.0500,"car, driver, passenger, city, fact, street, ex...",[],5,Great app!
40363,17.0,0.0618,"driver, rider, promo, rating, case, quality, a...",[super_convenient],5,Super convenient and fast!


#### 2. Mallet LDA - Find the most representative document for each topic

In [38]:
print("Find the most representative document for each topic")
sent_topics_sorteddf_m = find_most_representative_doc_for_each_doc(df_topic_sents_keywords_m, df)
sent_topics_sorteddf_m.style.set_properties(subset=['review'], **{'width': '400px'})

Find the most representative document for each topic


,Dominant_Topic,Perc_Contribution,Topic_Keywords,star_rating,review
0,0.000000,0.172600,"car, driver, passenger, city, fact, street, excuse, kind, turn, condition",1,Mehedi the driver was very unfriendly...Was driving the car recklessy over bumps..talking on the phone while driving.Body language was not acceptable picking his nose...when asked about the fare he started arguing saying he doesn't know and it is not his company.Telling me rudely he has another job to go to.The car was smelling full of cigarettes.The passenger seat was not in the upright position could hardly get my legs in..such a stressful journey!!!
1,1.000000,0.170500,"taxi, rate, day, application, app, fee, scooter, star, work, move",3,"Use this 2 times a day, very useful for me. Hower you need to fix 1 thing: release ( or better don't use at all) a WakeLock on ""Driving/Share"" and ""Rate"" activities cause it is so amazingly annoying. Here is the usual user work flow: order taxi -> swith screen off, use other apps etc -> get alert that taxi arrived -> click OK -> screen off, put it in the pocket and go to taxi (at this stage user DON'T care about app anymore) -> sit in the taxi -> driver starts driving -> and app silently lights up the screen in the pocket AND keep it awake eating obviously a battery -> you switch it off again -> at the end of driving app AGAIN wakes up the screen with rate activity oh c'mon... very annoying. And you need to add 1 optional textfiled below address for details to the driver. Otherwice all good"
2,2.000000,0.170300,"ride, easy_use, business, night, scam, town, voucher, book, offer, market",1,"I’ve had many problems with the customer support l, once the driver started screaming at me and demanding for me to leave the car! Although I ordered a bolt premium! The driver asked me to leave the vehicle and wouldn’t cancel the ride (if you have started the ride you as the customer can’t cancel it) I called the support they apologised and lied to me saying they will put 50£ worth of rides to my account. The next day my account was disabled, bolt demanded i provide evidence or they will dismiss this case. I didn’t get the money back for the unfinished horrible ride and no “50£” worth of credit. Another angering event was when the driver didn’t wait for me and just left starting the trip (as i said before this app does not provide a chance for the customer to cancel the ride once the driver has started it) it was also a bolt premium. I called the support multiple times, they have told me that they too can’t cancel the ride. The money has been deducted from my account AGAIN! DONT DOWNLOAD THIS APP!"
3,3.000000,0.207900,"customer, discount, charge, end, month, distance, morning, pay, care, arrival",1,I am a loyal and daily customer but lately the service I have received from your driver's is so not on. My kids also travel with me and most driver's do not apply to the road rules. And the 4 persent increase is not a fair increase..... I make use of this service every morning and most afternoons and the price I had to pay this morning was overpriced. Normally I would pay about R35 to R40 but this morning it was R55. Imagine I have to pay daily R110 just to travel from home to work and that is not even to drop off the kids at school or to pick them up from aftercare..... I would pay more than R3500 a month to just get from point a to point b. From now on I will rather make other arrangements
4,4.000000,0.159700,"time, minute, order, cab, pick, min, pickup, start, estimation, every_time",1,"Just stood outside for 15 minutes waiting for my driver. The time kept changing from 2 minutes to 4 minutes, to 3 minutes, back to 5 minutes. After 10 minutes I rang the driver who didn’t seem to even realise he’d taken the trip. He said he’s on his way and then canceled a few minutes later. Booked another who’s time also kept going up and down however this one actually did show up. Although due to the cancellation of the first trip I lost my discount 😡"
5,5

#### 3. Mallet LDA - Topic distribution across documents

In [39]:
# Show
print("Topic distribution across documents")
df_dominant_topic_m = topic_distribution_across_docs(df_topic_sents_keywords_m)
df_dominant_topic_m.style.set_properties(subset=['Keywords'], **{'width': '400px'})

Topic distribution across documents


,Dominant_Topic,Keywords,Num_Documents,Perc_Documents
0,0.000000,"car, driver, passenger, city, fact, street, excuse, kind, turn, condition",14582,0.361300
1,1.000000,"taxi, rate, day, application, app, fee, scooter, star, work, move",2622,0.065000
2,2.000000,"ride, easy_use, business, night, scam, town, voucher, book, offer, market",1785,0.044200
3,3.000000,"customer, discount, charge, end, month, distance, morning, pay, care, arrival",1651,0.040900
4,4.000000,"time, minute, order, cab, pick, min, pickup, start, estimation, every_time",1518,0.037600
5,5.000000,"driver, client, show, life, cancellation, rude, office, block, cancel_trip, communication",1458,0.036100
6,6.000000,"location, destination, option, map, place, address, point, pick_location, screen, everytime",1449,0.035900
7,7.000000,"issue, app, response, today, thing, update, experience, safety, yesterday, review",1447,0.035800
8,8.000000,"price, guy, reason, transportation, pricing, platform, purpose, alternative, range, demand",1445,0.035800
9,9.000000,"driver, work, experience, friend, area, vehicle, person, job, family, drop",1366,0.033800


In [34]:
coherence_model = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=dictionary, coherence='c_npmi')
coherence_model.get_coherence_per_topic()

[0.025636387283576208,
 -0.00033672387712726484,
 0.027459863226146547,
 0.0016659080521791824,
 -0.06243695078854283,
 0.022359884043397654]

# Topic Analysis

In [5]:
df_analysis = pd.read_csv(config['csv_input_local']['bolt_apple_google_p1_topic_sentiment'], index_col=0)
df_analysis

,Dominant_Topic,Perc_Contribution,Keywords,Text,rating,review,sentiment_score
0,0.0,0.0859,"car, driver, passenger, city, fact, street, ex...","['thing', 'prenook', 'trip', 'catch', 'flight'...",1,The first thing I noticed is that you can't pr...,-1
1,1.0,0.0924,"taxi, rate, day, application, app, fee, scoote...","['need', 'section', 'airport', 'spot', 'number...",4,Your GPS setting around Cape Town Internationa...,0
2,13.0,0.1102,"support, email, journey, complaint, reason, we...","['friend', 'use', 'would_allow', 'journey', 's...",1,I was invited by my friend and was given a £10...,-1
3,13.0,0.0776,"support, email, journey, complaint, reason, we...","['driver', 'support', 'computer', 'response', ...",3,Your drivers are great BUT your support is no ...,0
4,17.0,0.1186,"driver, rider, promo, rating, case, quality, a...","['rating', 'moment', 'driver', 'rating', 'trip...",3,"Loving the app, but can only give it a medium ...",1
...,...,...,...,...,...,...,...
40360,15.0,0.0711,"time, route, drivers_alway, drive, traffic, co...","['time', 'nonsense', 'driver', 'drive', 'suppo...",1,"Its the 5th time Im saying, But When Will ther...",0
40361,10.0,0.0686,"love, service, friendly_driver, nice_one, good...",['love'],5,Just love it!,3
40362,0.0,0.0500,"car, driver, passenger, city, fact, street, ex...",[],5,Great app!,3
40363,17.0,0.0618,"driver, rider, promo, rating, case, quality, a...",['super_convenient'],5,Super convenient and fast!,1


## Merge Similar Topics

In [16]:
df_analysis.loc[(df_analysis['Dominant_Topic'] == 2.0),'Dominant_Topic'] = 0.0
df_analysis.loc[(df_analysis['Dominant_Topic'] == 18.0),'Dominant_Topic'] = 1.0
df_analysis.loc[(df_analysis['Dominant_Topic'] == 8.0),'Dominant_Topic'] = 3.0
df_analysis.loc[(df_analysis['Dominant_Topic'] == 15.0),'Dominant_Topic'] = 4.0
df_analysis.loc[(df_analysis['Dominant_Topic'] == 11.0),'Dominant_Topic'] = 7.0
df_analysis.loc[(df_analysis['Dominant_Topic'] == 16.0),'Dominant_Topic'] = 13.0
df_analysis.loc[(df_analysis['Dominant_Topic'] == 19.0),'Dominant_Topic'] = 17.0

In [19]:
df_analysis[df_analysis['Dominant_Topic'] == 17.0]

,Dominant_Topic,Perc_Contribution,Keywords,Text,rating,review,sentiment_score
4,17.0,0.1186,"driver, rider, promo, rating, case, quality, a...","['rating', 'moment', 'driver', 'rating', 'trip...",3,"Loving the app, but can only give it a medium ...",1
6,17.0,0.0845,"driver, rider, promo, rating, case, quality, a...","['promotion', 'price', 'driver', 'rider', 'pri...",3,I like the promotion price. But some of the dr...,-3
14,17.0,0.0703,"service, driver, request, promotion, app, hour...","['driver', 'driver', 'thing', 'customer', 'ser...",1,I requested a ride and the driver (Akhona) com...,-2
22,17.0,0.0762,"driver, rider, promo, rating, case, quality, a...","['driving', 'driver', 'driver', 'bill', 'bill'...",4,Its always fun driving with bolt because you a...,0
23,17.0,0.0649,"service, driver, request, promotion, app, hour...","['figure', 'destination', 'driver', 'amount', ...",2,Inconsistent and unpredictable. When you are r...,-1
...,...,...,...,...,...,...,...
40222,17.0,0.0636,"service, driver, request, promotion, app, hour...","['promotion', 'account', 'chat', 'confirmation...",1,With the „10€ promotion“ still more expensive ...,-4
40237,17.0,0.0856,"service, driver, request, promotion, app, hour...","['driver', 'ride', 'stall', 'move', 'cancel_ri...",1,The taxi drivers come as they please. If they ...,0
40326,17.0,0.0673,"driver, rider, promo, rating, case, quality, a...",['rating'],2,The app crashes after giving the rating.,0
40351,17.0,0.0615,"service, driver, request, promotion, app, hour...",['top'],5,Top-top,0
